This notebook notebook runs a full grid evaluation of the E-field distributional landscape given input parameters $(x,y,\theta)$. This is achieved massive experiment parallelization using sbatch then collection into a local directory via file deposition.  

In [1]:
import sys

In [2]:
#SCC package environment
sys.path.insert(0,'/KIMEL/tigrlab/projects/jjeyachandra/gmsh-sdk/lib/')
sys.path.insert(0,'/projects/jjeyachandra/simnibs/miniconda2/envs/simnibs_env/lib/python2.7/site-packages')
sys.path.insert(0,'/projects/jjeyachandra/simnibs/Python_modules/src')

In [6]:
from shutil import copy

In [3]:
#Load custom slurm magic
%load_ext slurm_magic

In [41]:
%%sbatch
#!/usr/bin/env python
#SBATCH --partition=low-moby
#SBATCH --output=/projects/jjeyachandra/rtms_optimize/objective_grid/log/sim_%a.out
#SBATCH --error=/projects/jjeyachandra/rtms_optimize/objective_grid/log/sim_%a.err
#SBATCH --nodes=1
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=5G
#SBATCH --job-name=field_simulations
#SBATCH --array=0-19

#Package import
import sys
sys.path.insert(0,'/projects/jjeyachandra/simnibs/miniconda2/envs/simnibs_env/lib/python2.7/site-packages')
sys.path.insert(0,'/projects/jjeyachandra/simnibs/Python_modules/src')

import os
from shutil import copy

import numpy as np
from simnibs import sim_struct, run_simulation
from fieldopt import geolib, tetrapro

####### DATA LOADING

#File specification
proj_dir = '/projects/jjeyachandra/rtms_optimize'
C = np.fromfile(os.path.join(proj_dir,'output','quadratic_vec'))
iR = np.fromfile(os.path.join(proj_dir,'output','inverse_rot')).reshape(3,3)
b = np.fromfile(os.path.join(proj_dir,'output','param_bounds')).reshape(3,2)
grid_path = os.path.join(proj_dir,'objective_grid','grid')

###### INPUT SETUP 

#Set up evaluation grid
fidelity = 20
x_samps = np.linspace(b[0,0],b[0,1],fidelity)
y_samps = np.linspace(b[1,0],b[1,1],fidelity)
rot_samps = np.linspace(0,180,fidelity)
XX,YY,RR = np.meshgrid(x_samps,y_samps,rot_samps)
XX = XX.flatten().reshape((fidelity**3,1))
YY = YY.flatten().reshape((fidelity**3,1))
RR = RR.flatten().reshape((fidelity**3,1))
sample_set = np.concatenate((XX,YY,RR),axis=1)

#Pick input parameters
ind = int(os.environ['SLURM_ARRAY_TASK_ID'])
x,y,r = sample_set[ind,:] 

#Set up spatial postioning
preaff_loc = geolib.map_param_2_surf(x,y,C)
loc = np.matmul(iR,preaff_loc)

#Set up rotational positioning
preaff_rot, preaff_n = geolib.map_rot_2_surf(x,y,r,C)
rot = np.matmul(iR,preaff_rot)
n = np.matmul(iR,preaff_n)

###### SIMULATION SETUP

#Start a session
S = sim_struct.SESSION()
S.fnamehead = os.path.join(proj_dir,'data','simnibs_output','sub-CMH090.msh')
S.pathfem = os.path.join(grid_path,'sim_{}'.format(ind))
tms = S.add_tmslist()
tms.fnamecoil='/projects/jjeyachandra/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz'
pos = tms.add_position()
pos.matsimnibs = geolib.define_coil_orientation(loc,rot,n)

print('Running simulation with:')
print('INPUTS: {} {} {}'.format(x,y,r))
print('INDEX: {}'.format(ind))
print('POSITION ARRAY:')
print(pos.matsimnibs)

#Run simulation
run_simulation(S)
print('Finished successfully!')

###### PROCESS OUTPUTS 

#Save the geo file
geo_dir=os.path.join(proj_dir,'objective_grid','geo_output')
geo_file=os.path.join(grid_path,'sim_{}'.format(ind),
                      'sub-CMH090_TMS_1-0001_Magstim_70mm_Fig8_nii_coil_pos.geo')
geo_out=os.path.join(geo_dir,'sub-CMH090_{}.geo'.format(ind))
copy(geo_file,geo_out)

#Compute score on output
field_file=os.path.join(S.pathfem,'sub-CMH090_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh')
parcelmap_path = os.path.join(proj_dir,'output','tetra_parcels')
num_parcels = 20
target_parcel = 2
proj_map = np.fromfile(parcelmap_path)
proj_map = proj_map.reshape( (proj_map.shape[0]//(num_parcels + 1), num_parcels + 1) )
_, elem_ids, _ = geolib.load_gmsh_elems(S.fnamehead,entity=(3,2))
norm_E = geolib.get_field_subset(field_file,elem_ids)
score = geolib.compute_field_score(norm_E,proj_map,target_parcel)

#Save score value
score_dir=os.path.join(proj_dir,'objective_grid','scores')
score_file=os.path.join(score_dir,'fx_{}'.format(ind))
np.save(score_file,score)

print('Finished successfully')

u'Submitted batch job 65820\n'